In [1]:
import pandas as pd

def load_data(path: str):
  years = list(range(2005, 2020))
  year2index = { y : i for (i, y) in enumerate(years) }
  data = [pd.read_excel(path, sheet_name=str(y)) for y in years]
  return data, years, year2index

data, years, year2index = load_data("./data/Brazilian Immigrants - modified.xlsx")
data[year2index[2005]].head()

,Type,Unnamed: 1,State Code,1,2,4,5,6,8,9,...,41,42,44,45,47,48,49,51,53,55
0,Population,NaN,Population,2030.0,88,1288.0,861.0,25111.0,1339.0,14021.0,...,571.0,7019.0,1143.0,2339.0,1355.0,7921.0,2904.0,6471.0,2505.0,804.0
1,Age,2.0,Age 0 to 4,78.0,0,0.0,0.0,305.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,20.0,0.0,87.0,0.0,0.0
2,Age,4.0,Age 05 to 09,172.0,0,0.0,116.0,1039.0,84.0,333.0,...,0.0,157.0,0.0,0.0,0.0,283.0,182.0,1180.0,0.0,85.0
3,Age,14.5,Age 10 to 19,242.0,0,0.0,0.0,2920.0,0.0,1309.0,...,0.0,748.0,0.0,119.0,0.0,382.0,360.0,439.0,467.0,213.0
4,Age,22.0,Age 20 to 24,0.0,0,67.0,0.0,1910.0,0.0,2153.0,...,120.0,795.0,75.0,577.0,0.0,831.0,154.0,394.0,97.0,0.0


In [2]:
def load_state_dict(path: str):
  data = pd.read_excel(path, sheet_name="State code")
  state_dict = dict()

  for i in data.index:
    abbrev = data["State Abbreviation"][i].lower()
    name = data["State Name"][i].lower()
    code = data["FIPS Code"][i]
    
    state_dict[code] = {
      "name": name,
      "abbrev": abbrev
    }

  return state_dict

state_dict = load_state_dict("./data/Brazilian Immigrants.xlsx")
state_dict

{1.0: {'name': 'alabama', 'abbrev': 'al'},
 2.0: {'name': 'alaska', 'abbrev': 'ak'},
 4.0: {'name': 'arizona', 'abbrev': 'az'},
 5.0: {'name': 'arkansas', 'abbrev': 'ar'},
 6.0: {'name': 'california', 'abbrev': 'ca'},
 8.0: {'name': 'colorado', 'abbrev': 'co'},
 9.0: {'name': 'connecticut', 'abbrev': 'ct'},
 10.0: {'name': 'delaware', 'abbrev': 'de'},
 11.0: {'name': 'district of columbia', 'abbrev': 'dc'},
 12.0: {'name': 'florida', 'abbrev': 'fl'},
 13.0: {'name': 'georgia', 'abbrev': 'ga'},
 15.0: {'name': 'hawaii', 'abbrev': 'hi'},
 16.0: {'name': 'idaho', 'abbrev': 'id'},
 17.0: {'name': 'illinois', 'abbrev': 'il'},
 18.0: {'name': 'indiana', 'abbrev': 'in'},
 19.0: {'name': 'iowa', 'abbrev': 'ia'},
 20.0: {'name': 'kansas', 'abbrev': 'ks'},
 21.0: {'name': 'kentucky', 'abbrev': 'ky'},
 22.0: {'name': 'louisiana', 'abbrev': 'la'},
 23.0: {'name': 'maine', 'abbrev': 'me'},
 24.0: {'name': 'maryland', 'abbrev': 'md'},
 25.0: {'name': 'massachusetts', 'abbrev': 'ma'},
 26.0: {'name':

In [14]:
def query_by_attr_year(attr: str, year: int):
  df = data[year2index[year]]
  return df.loc[df.iloc[:, 0] == attr]

# row = query_by_attr_year("Average Age", 2006)
row = query_by_attr_year("Proportion", 2006)
row.head()

,Type,Unnamed: 1,State Code,1,4,5,6,8,9,10,...,45,46,47,48,49,50,51,53,54,55
21,Proportion,NaN,Female,0.613139,0.439024,0,0.586026,0.5768,0.4424,0.640523,...,0.527467,0.0,0.542497,0.579614,0.588071,0.638889,0.539957,0.618755,0.55914,0.958057
22,Proportion,NaN,Male,0.386861,0.560976,1,0.413974,0.4232,0.5576,0.359477,...,0.472533,1.0,0.457503,0.420386,0.411929,0.361111,0.460043,0.381245,0.44086,0.041943


In [39]:
def query_by_attr_year_state(attr: str, year: int, state: int):
  row = query_by_attr_year(attr, year)
  value = np.around(row[state].to_numpy(), decimals=2) if state in row.columns else ([None] * len(row))
  return value, row.iloc[:, 2].to_list()

# print(query_by_attr_year_state("Average Age", 2006, state=1))
print(query_by_attr_year_state("Proportion", 2006, state=1))

(array([0.61, 0.39]), ['Female', 'Male'])


In [40]:
import numpy as np

def query_attr_over_years_by_state(attr: str, state: int):
  values = []

  for y in years:
    value, attrNames = query_by_attr_year_state(attr, y, state)
    values.append(value)
  
  return np.asarray(values).T, attrNames

# print(query_attr_over_years_by_state("Average Age", state=1))
print(query_attr_over_years_by_state("Proportion", state=1))

(array([[0.47, 0.61, 0.32, 0.37, 0.71, 0.69, 0.64, 0.  , 0.75, 0.7 , 0.54,
        0.61, 0.38, 0.85, 0.62],
       [0.53, 0.39, 0.68, 0.63, 0.29, 0.31, 0.36, 1.  , 0.25, 0.3 , 0.46,
        0.39, 0.62, 0.15, 0.38]]), ['Female', 'Male'])


In [41]:
from typing import List, Optional
import pyecharts.options as opts
from pyecharts.charts import Grid, Line
import os

def plot_proportion_over_years_of_states(
  state: int, 
  save_name: Optional[str] = None
):
  values_by_state, attr_names = query_attr_over_years_by_state("Proportion", state)
  state_name = state_dict[state]["name"].title()

  line = (
    Line()
    .add_xaxis(xaxis_data=[str(y) for y in years])
    .set_global_opts(
      title_opts=opts.TitleOpts(
        title=f"Gender Population Proportion of {state_name} States"
      ),
      tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross"),
      yaxis_opts=opts.AxisOpts(
        type_="value",
        axistick_opts=opts.AxisTickOpts(is_show=True),
        splitline_opts=opts.SplitLineOpts(is_show=True)
      ),
      datazoom_opts=opts.DataZoomOpts(
        range_start=0,
        range_end=100
      ),
      legend_opts=opts.LegendOpts(is_show=True, pos_top=40)
    )
  )

  for (i, values) in enumerate(values_by_state):
    line.add_yaxis(
      series_name=attr_names[i],
      y_axis=values,
      label_opts=opts.LabelOpts(is_show=False)
    )

  grid = (
    Grid()
      .add(line, grid_opts=opts.GridOpts(pos_top=80))
  ) 

  if save_name is not None:
    grid.render(os.path.join("./state/proportion", save_name + ".html"))

  return grid

plot_proportion_over_years_of_states(
  state=48, 
  save_name="proportion_tx"
).render_notebook()

In [42]:
for s in list(state_dict.keys()):
  plot_proportion_over_years_of_states(
    state=s, 
    save_name="proportion_" + state_dict[s]["abbrev"]
  )

In [48]:
def plot_ages_over_years_by_states(
  states: List[int], 
  save_name: Optional[str] = None
):
  values_by_state = [query_attr_over_years_by_state("Average Age", state)[0].reshape(-1) for state in states]

  line = (
    Line()
    .add_xaxis(xaxis_data=[str(y) for y in years])
    .set_global_opts(
      title_opts=opts.TitleOpts(
        title=f"Average Age of {len(states)} States"
      ),
      tooltip_opts=opts.TooltipOpts(trigger="axis"),
      yaxis_opts=opts.AxisOpts(
        type_="value",
        axistick_opts=opts.AxisTickOpts(is_show=True),
        splitline_opts=opts.SplitLineOpts(is_show=True)
      ),
      datazoom_opts=opts.DataZoomOpts(
        range_start=0,
        range_end=100
      ),
      legend_opts=opts.LegendOpts(is_show=True, pos_top=40)
    )
  )

  for (i, values) in enumerate(values_by_state):
    line.add_yaxis(
      series_name = state_dict[states[i]]["abbrev"].upper(),
      y_axis=values,
      label_opts=opts.LabelOpts(is_show=False),
    )

  grid = (
    Grid(init_opts=opts.InitOpts(width="775px", height="600px"))
      .add(line, grid_opts=opts.GridOpts(pos_top=200))
  ) 

  if save_name is not None:
    grid.render(os.path.join("./state/avg_age", save_name + ".html"))

  return grid

plot_ages_over_years_by_states(
  states=list(state_dict.keys()),
  save_name="avg_age"
).render_notebook()